In [1]:
import sys
sys.path.insert(0, "./src")

import f5_tts.model
print("f5_tts.model contents:", dir(f5_tts.model))


/home/mike/github/wrightmikea/DMOSpeech2/.venv/lib/python3.10/site-packages/jieba/_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.551 seconds.
Prefix dict has been built successfully.


Word segmentation module jieba initialized.



/home/mike/github/wrightmikea/DMOSpeech2/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


f5_tts.model contents: ['CFM', 'DiT', 'MMDiT', 'Trainer', 'UNetT', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'backbones', 'cfm', 'dataset', 'modules', 'trainer', 'utils']


In [2]:
import duration_predictor
print("duration_predictor contents:", dir(duration_predictor))


duration_predictor contents: ['PositionalEncoding', 'SpeechLengthPredictor', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'calculate_remaining_lengths', 'nn', 'torch']


In [3]:
import guidance_model
print("guidance_model contents:", dir(guidance_model))


guidance_model contents: ['Callable', 'ConformerCTC', 'ConformerDiscirminator', 'DiT', 'ECAPA_TDNN', 'F', 'Guidance', 'NoOpContext', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_kl_dist_func', 'annotations', 'default', 'exists', 'lens_to_mask', 'list_str_to_idx', 'list_str_to_tensor', 'mask_from_frac_lengths', 'nn', 'np', 'predict_flow', 'random', 'torch']


In [4]:
import os
print("Current directory explicitly is:", os.getcwd())
#print("Explicit contents of ckpts folder:", os.listdir('ckpts'))

Current directory explicitly is: /home/mike/github/wrightmikea/DMOSpeech2/src


In [ ]:
import sys
sys.path.insert(0, "./src")

from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import FileResponse, JSONResponse
import torchaudio
import torch
import uuid
import nest_asyncio
import uvicorn

from infer import DMOInference

nest_asyncio.apply()

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize the DMO inference model (matching working dmo_tts_api.py)
tts = DMOInference(
    student_checkpoint_path="../ckpts/model_85000.pt",
    duration_predictor_path="../ckpts/model_1500.pt",
    device=DEVICE,
    model_type="F5TTS_Base"
)

app = FastAPI()

# Explicit state storage for reference voice (matching working pattern)
style_state = {"prompt_audio": None, "prompt_text": None}

@app.post("/init_voice")
async def init_voice(audio_file: UploadFile = File(...), reference_text: str = Form(...)):
    audio_data, sr = torchaudio.load(audio_file.file)
    audio_path = f"/tmp/{uuid.uuid4().hex}_ref.wav"
    torchaudio.save(audio_path, audio_data, sr)
    
    style_state["prompt_audio"] = audio_path
    style_state["prompt_text"] = reference_text
    
    return JSONResponse({"status": "Voice style initialized.", "audio_saved": audio_path})

@app.post("/generate_audio")
async def generate_audio(target_text: str = Form(...)):
    if style_state["prompt_audio"] is None or style_state["prompt_text"] is None:
        return JSONResponse({"error": "Initialize voice first."}, status_code=400)

    with torch.no_grad():
        generated_audio = tts.generate(
            gen_text=target_text,
            audio_path=style_state["prompt_audio"],
            prompt_text=style_state["prompt_text"]
        )

    audio_tensor = torch.from_numpy(generated_audio).unsqueeze(0)
    filename = f"/tmp/{uuid.uuid4().hex}_gen.wav"
    torchaudio.save(filename, audio_tensor, 24000)

    return FileResponse(filename, media_type='audio/wav', filename="generated.wav")

uvicorn.run(app, host="0.0.0.0", port=8000)

Download Vocos from huggingface charactr/vocos-mel-24khz


INFO:     Started server process [17682]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     192.168.1.246:58189 - "POST /init_voice HTTP/1.1" 200 OK
Converting audio...
Using custom reference text...

ref_text   Is ant it ironic?. 
INFO:     192.168.1.246:58244 - "POST /generate_audio HTTP/1.1" 200 OK
